<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5 - Prompting Large Language Models

In this lab we will practise prompting with a few Large Language Models (LLMs) using Groq (not to be confused with Grok). Groq is a platform that provides access to their custom-built AI hardware via APIs, allowing users to run open-source models such as Llama.

We shall see that while LLMs are powerful tools, how you ask a question or frame a task can dramatically influence the results obtained.

## Set-up

Step 1: Sign up for a free Groq account at https://console.groq.com/home .

Step 2: Create a new API key at https://console.groq.com/keys. Copy-paste it into an empty text file called 'groq_key.txt'.

Running the next cell will then read in this key and assign it to the variable `groq_key`.

In [84]:
groqfilename = r'/Users/annaxu/Documents/Data Science/Module 8/Labs/groq_key.txt' # this file contains a single line containing your Groq API key only
try:
    with open(groqfilename, 'r') as f:
        groq_key = f.read().strip()
except FileNotFoundError:
    print("'%s' file not found" % filename)

In [86]:
!pip install groq

In [87]:
from groq import Groq
import requests
import pandas as pd
from IPython.display import Markdown

First create an instance of the Groq client:

In [89]:
client = Groq(api_key=groq_key)

The following code shows what models are currently accessible through Groq. `context_window` refers to the size of memory (in tokens) during a session and `max_completion_tokens` is the maximum number of tokens that are generated in an output.

In [94]:
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {groq_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data']).sort_values(['created'], ascending=False)

,id,object,created,owned_by,active,context_window,public_apps,max_completion_tokens
18,meta-llama/llama-prompt-guard-2-86m,model,1748632165,Meta,True,512,None,512
1,meta-llama/llama-prompt-guard-2-22m,model,1748632101,Meta,True,512,None,512
9,qwen/qwen3-32b,model,1748396646,Alibaba Cloud,True,131072,None,40960
10,meta-llama/llama-guard-4-12b,model,1746743847,Meta,True,131072,None,1024
7,meta-llama/llama-4-maverick-17b-128e-instruct,model,1743877158,Meta,True,131072,None,8192
17,meta-llama/llama-4-scout-17b-16e-instruct,model,1743874824,Meta,True,131072,None,8192
16,compound-beta-mini,model,1742953279,Groq,True,131072,None,8192
20,qwen-qwq-32b,model,1741214760,Alibaba Cloud,True,131072,None,131072
11,compound-beta,model,1740880017,Groq,True,131072,None,8192
5,playai-tts-arabic,model,1740682783,PlayAI,True,8192,None,8192


The Groq client object enables interaction with the Groq REST API and a chat completion request is made via the client.chat.completions.create method.

The most important arguments of the client.chat.completions.create method are the following:
* messages: a list of messages (dictionary form) that make up the conversation to date
* model: a string indicating which model to use (see [list of models](https://console.groq.com/docs/models))
* max_completion_tokens: the maximum number of tokens that are generated in the chat completion
* response_format: setting this to `{ "type": "json_object" }` enables JSON output
* seed: sample deterministically as best as possible, though identical outputs each time are not guaranteed
* temperature: between 0 and 2 where higher values like 0.8 make the output more random (creative) and values like 0.2 are more focused and deterministic


In [96]:
help(client.chat.completions.create)

Help on method create in module groq.resources.chat.completions:

create(*, messages: 'Iterable[ChatCompletionMessageParam]', model: "Union[str, Literal['gemma2-9b-it', 'llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'llama-guard-3-8b', 'llama3-70b-8192', 'llama3-8b-8192']]", exclude_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, function_call: 'Optional[completion_create_params.FunctionCall] | NotGiven' = NOT_GIVEN, functions: 'Optional[Iterable[completion_create_params.Function]] | NotGiven' = NOT_GIVEN, include_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN, max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, metadata: 'Optional[Dict[str, str]] | NotGiven' = NOT_GIVEN, n: 'Optional[int] | NotGiven' = NOT_GIVEN, parallel_tool_calls:

As a first example, note how the messages input is given as a list of a dictionaries with `role` and `content` keys. This is in a ChatML format recognised by many LLMs.

In [100]:
chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system", # sets the persona of the model
            "content": "You are a helpful assistant."
        },
        {
            "role": "user", # what the user wants the assistant to do
            "content": "Explain briefly how large language models work",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training**: Being trained on massive amounts of text data, which helps them learn patterns and relationships in language.
2. **Tokenization**: Breaking down text into small units called tokens, such as words or characters.
3. **Contextualization**: Analyzing the context in which tokens are used to understand their meaning.
4. **Prediction**: Using complex algorithms to predict the next token or generate text based on the context.

This process allows large language models to learn, understand, and generate human-like language, enabling applications such as text generation, language translation, and conversational AI.


The output is in Markdown format so the following line formats this text.

In [102]:
Markdown(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training**: Being trained on massive amounts of text data, which helps them learn patterns and relationships in language.
2. **Tokenization**: Breaking down text into small units called tokens, such as words or characters.
3. **Contextualization**: Analyzing the context in which tokens are used to understand their meaning.
4. **Prediction**: Using complex algorithms to predict the next token or generate text based on the context.

This process allows large language models to learn, understand, and generate human-like language, enabling applications such as text generation, language translation, and conversational AI.

## Text summarisation

We start with a llama3-8b-8192, a model using just over 8 billion parameters with at most 8192 tokens produced as output.

Here is an article to be summarised from the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset:

In [108]:
story = """
SAN FRANCISCO, California (CNN) -- A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.
"""

**Exercise:**
Summarise the story text using the following three prompts. Use the format given above but here there is no need to set the persona (i.e. only include one dictionary in the messages list when calling `client.chat.completions.create`.) Comment on any differences.

1) "Summarise the following article in 3 sentences."

2) "Give me a TL;DR of this text."

3) "What's the key takeaway here?"

In [111]:
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

for p in prompts:
    response = client.chat.completions.create(
                messages=[{"role": "user", "content": p + story}],
                model="llama3-8b-8192"
)

    print('-'*50, '\n', p, '\n', response.choices[0].message.content)

-------------------------------------------------- 
 Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area on Friday morning, with its epicenter located about two miles east-northeast of Oakland. The quake caused minimal damage and only briefly disrupted power for about 2,000 customers, with no reported injuries or significant damage. The shaking lasted around 50 seconds and was described as "fairly mild" by authorities, with most calls to emergency services being inquiries rather than reports of damage or injury.
-------------------------------------------------- 
 Give me a TL;DR of this text.  
 Here is a brief summary of the text:

A magnitude 4.2 earthquake struck the San Francisco area at 4:42 a.m. local time, causing minimal damage and power outages for about 2,000 customers. The quake was considered "light" and lasted around 50 seconds, with reports of no injuries or signific

Run the above code again below and note that the answers may differ. This is due to the probabilistic nature of LLM token generation.

In [113]:
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

for p in prompts:
    response = client.chat.completions.create(
                messages=[{"role": "user", "content": p + story}],
                model="llama3-8b-8192"
)

    print('-'*50, '\n', p, '\n', response.choices[0].message.content)

-------------------------------------------------- 
 Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area at 4:42 a.m. on Friday, shaking for about 50 seconds and leaving around 2,000 customers without power. The US Geological Survey classified the earthquake as "light", meaning it usually causes minimal damage, and initial reports showed no injuries or damage reported to authorities. The quake was centered near Oakland, east of San Francisco, and while it may have caused some disturbance, it was described as "fairly mild" by local officials.
-------------------------------------------------- 
 Give me a TL;DR of this text.  
 A magnitude 4.2 earthquake struck the San Francisco area at 4:42am, causing about 2,000 customers to lose power, but no injuries or major damage were reported. The quake, considered "light" by the USGS, was centered near Oakland and lasted about 50 seconds, wi

## Text completion

**Exercise**: In this section adjust the `max_completion_tokens` and `temperature` settings below to obtain different responses. Show some examples with the prompt "Continue the story: It was a great time to be alive" with the model "llama-3.1-8b-instant".

* max_completion_tokens - the maximum number of tokens to generate. Note that longer words are made of multiple tokens (set to 200 and 500)
* temperature (positive number) - the higher the number the more random (creative) the output (set to 0.2, 0.8, 2)

In [118]:
# ANSWER (set max_completion_tokens=200, do not have a temperature setting)
prompt = "Continue the story: It was a great time to be alive"

response = client.chat.completions.create(
    messages = [{"role": "user", "content": prompt}],
    max_completion_tokens = 200,
    model = "llama3-8b-8192"
)

print(response.choices[0].message.content)

It was a great time to be alive. The sun was shining, the birds were singing, and the air was filled with the sweet scent of blooming flowers. People were laughing and smiling, eager to start their day. It was a time of hope and opportunity, a time when anything seemed possible.

As you walked down the street, you couldn't help but feel the energy of the city. The streets were bustling with activity, people rushing to and fro, heading to work or school or on a day out. The sound of construction echoed through the air, as new buildings and roads were being built to keep up with the growing population.

It was a time of great technological advancements. Cars, trains, and planes zoomed through the air, carrying people and goods with ease. And with the help of computers and the internet, people were connected like never before.

But it wasn't just the technology that made this time great. It was also the sense of community that thrived. People came


In [120]:
# ANSWER (set max_completion_tokens=500, do not have a temperature setting)
response = client.chat.completions.create(
    messages = [{"role": "user", "content": prompt}],
    max_completion_tokens = 500,
    model = "llama3-8b-8192"
)

print(response.choices[0].message.content)

It was a great time to be alive. The year was 2050, and the world was vastly different from the one we knew just a few decades prior. Cities were now sprawling metropolises with towering skyscrapers that seemed to touch the sky, and the air was crisp and clean, thanks to the widespread adoption of advanced air purification systems.

Humanity had also made tremendous strides in medicine, with diseases like cancer and Alzheimer's relegated to the history books. People were living longer, healthier lives, and the quality of life had improved dramatically.

But it wasn't all sunshine and rainbows. There were still challenges to be faced, of course. Climate change remained a pressing issue, and there were still pockets of poverty and inequality scattered throughout the globe.

Despite these remaining problems, the general mood was one of optimism and excitement. The world was changing at an incredible pace, and people were eager to see what the future held.

As I walked through the city, I 

In [122]:
# ANSWER (set temperature = 0.2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    messages = [{"role": "user", "content": prompt}],
    temperature = 0.2, #more focused and deterministic
    model = "llama3-8b-8192"
)

print(response.choices[0].message.content)

It was a great time to be alive. The sun was shining, the birds were singing, and the world seemed to be full of endless possibilities. I was 25 years old, with a bright future ahead of me. I had just landed my dream job at a prestigious tech company, and I was excited to start my new role.

As I walked to work, I couldn't help but feel a sense of pride and accomplishment. I had worked hard to get to this point, and it felt amazing to know that all my efforts had paid off. The city was bustling with energy, and I felt like I was a part of it all.

When I arrived at the office, my colleagues welcomed me with open arms. They were a talented and diverse group of people, and I was excited to learn from them and contribute to the team. Our company was known for its innovative approach to technology, and I was eager to be a part of it.

As the day went on, I found myself getting more and more excited about my new role. I was given a lot of responsibility, and I was determined to make the mos

In [125]:
# ANSWER (set temperature = 1, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    messages = [{"role": "user", "content": prompt}],
    temperature = 1, #semi-random
    model = "llama3-8b-8192"
)

print(response.choices[0].message.content)

As I gazed out at the bustling city, I couldn't help but feel a sense of excitement and possibility. The sun was shining, the air was crisp, and the smell of fresh-cut grass wafted through the streets. It was a great time to be alive, and I felt grateful to be experiencing it.

I was sitting in a small café, sipping on a latte and watching the world go by. The sounds of chatter and laughter filled the air, and I couldn't help but join in. I felt a sense of community and connection with the people around me, and it made me feel grateful to be a part of this world.

As I sat there, I couldn't help but think about all the amazing things that were happening in the world right now. New scientific discoveries were being made, new technologies were being developed, and new forms of art and music were emerging. It was a time of great change and progress, and I felt lucky to be a part of it.

But as I sat there, lost in my own thoughts, I couldn't help but feel a sense of unease. There were sti

Note what happens when the temperature is set too high!

In [127]:
# ANSWER (set temperature = 2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    messages = [{"role": "user", "content": prompt}],
    temperature = 2, #randomest
    model = "llama3-8b-8192"
)

print(response.choices[0].message.content)

It was a great time to be alive. Joey leaned back in his creaky office chair, swivel-twisting around in its worn canvas coating as he perched unsteadily between enthusiasm and trepidation. He wasn't certain, some twenty years future-looking-out from this moment in high school commencement just taken, what spectaculaes tomorrow would rustling forth, how some others close by he now huzz-o' it all -- though this '67 America in varevitch.

Some would remark `what nosh shucks! FUTURE`. New 'gaining air and wobble, into shivers here ! It a, this – so life-ruffling events pander well wharfly, once only us re-stold this epoch…’ You need feel too a new: better to – just but most and to…to where: you no-one new again a time.
Even more were the 2 who gazed each after's, just and.
Had Joey never or given when time in but -- just his better choice.

Citing few were, and, after — then no ‘em both: said hazy.
but — So ‘unquiet ' it b in what I from of as, n .
Tears I like from another their bough -'!

### Zero-shot and one-short prompting for question-answering

This section shows the impact of prompting on the response. Zero-shot prompting means we provide the prompt without any examples or additional context. Let us initially ask Mistral a question using no prompting.

In [129]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

The way two chemicals react depends on the specific chemical properties and the conditions in which they interact. Here are the general steps involved in a chemical reaction:

1. **Collision**: Two molecules (atoms or ions) must collide with each other. The likelihood of a collision depends on the concentration and movement of the molecules.
2. **Orientation**: The molecules must be oriented in a way that allows them to interact. This is influenced by the shape and polarity of the molecules.
3. **Energy**: The molecules must have enough energy to overcome the repulsive forces between them and form a new bond.
4. **Reaction**: If the molecules meet the above conditions, a chemical reaction occurs, resulting in the formation of new products.

There are several types of chemical reactions, including:

1. **Synthesis**: Two or more molecules combine to form a new molecule.
Example: 2H2 + O2 → 2H2O
2. **Decomposition**: A single molecule breaks down into simpler molecules.
Example: 2H2O → 2H2 + O2
3. **Single displacement**: One element displaces another element from a compound.
Example: Zn + CuSO4 → ZnSO4 + Cu
4. **Double displacement**: Two compounds exchange partners.
Example: NaCl + AgNO3 → NaNO3 + AgCl
5. **Combustion**: A molecule reacts with oxygen to release energy.
Example: CH4 + 2O2 → CO2 + 2H2O

To predict how two chemicals will react, you can use the following factors:

1. **Electronegativity**: The tendency of an atom to attract electrons.
2. **Polarity**: The distribution of electrons within a molecule.
3. **Reactivity series**: A list of elements arranged by their reactivity.
4. **Acid-base properties**: The ability of a molecule to donate or accept a proton.

It's essential to note that predicting chemical reactions involves a complex interplay of these factors and requires a strong understanding of chemistry principles.

**Exercise:** Ask the same question but modify the prompt to return the answer to the same question in a simpler form (still using the llama-3.1-8b-instant model). Experiment with different prompts.

In [139]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Explain in 50 words or less: How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

Chemical reactions occur when two chemicals interact, resulting in a change in their structure. This change is usually driven by the transfer of electrons, bonds breaking, or forming new ones. Energy is often released or absorbed in the process, leading to a new compound or products with different properties.

### One-shot prompting ###

Next, note the dramatic change when we give the following template setting a new role and providing an English question followed by a French translation.

In [148]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "system",
             "content": "You translate English to French."},
              {"role": "user",
               "content": "What time is it?"}, #this time example provides a sample so the model can follow the pattern
               {"role": "assistant",
               "content": "Quelle heure est-il?"},
              {"role": "user",
               "content": "How do two chemicals react?"}],
    temperature = 0.8,
)
print(response.choices[0].message.content)

Comment deux substances chimiques réagissent-elles?


### Few-shot prompting

Recall that since the text generation process outputs one token at a time, their outputs often need adjusting. This is where examples can help.

In [150]:
prompt1 = "I'm gonna head out now, see you later."
response1 = "I will be leaving now. See you later."

prompt2 =  "That movie was super cool!"
response2 = "The movie was very impressive."

prompt3 = "Can't make it to the meeting, sorry."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a professional editor. Rewrite casual sentences into a formal tone."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


Regrettably, I will be unable to attend the meeting. I apologize for any inconvenience this may cause.


The output can also be moulded to provide SQL output.

In [152]:
prompt1 = "Show me all users who signed up in the last 30 days."
response1 = "SELECT * FROM users WHERE signup_date >= CURRENT_DATE - INTERVAL '30 days';"

prompt2 = "What is the average order value?"
response2 =  "SELECT AVG(order_total) FROM orders;"

prompt3 = "List products that are out of stock."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are an assistant that translates natural language to SQL."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())

SELECT * FROM products WHERE quantity = 0;


**Exercise**: Create a few examples to train the "llama3-70b-8192" LLM to take in user content in the form below and provide output as a pandas dataframe. Use the `exec` function to execute its output to display the answer of sample input as a data frame.

Example:

given the user content

"""

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

train the model to output

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})



In [202]:
prompt1 = """

| col1 | col2 | col3

| 40 | 7 | 27

| 10 | 48 | 15

"""
response1 = "df = pd.DataFrame({'col1': [40, 10], 'col2': [7, 28], 'col3': [27, 15]})"

prompt2 = """

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a data scientist, provide single code line to convert input into a pandas dataframe dictionary"},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2}
    ]
)

print(response.choices[0].message.content.strip())

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})


Also show what happens when the question is asked in the absence of a system role and without few-shot prompting.

In [208]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "user", "content": prompt2},
    ]
)

print(response.choices[0].message.content.strip())

It looks like you're trying to display a table with three columns (`col1`, `col2`, and `col3`) and two rows of data.

Here's a formatted version of your table:

| col1 | col2 | col3 |
| --- | --- | --- |
| 32  | 27  | 25  |
| 64  | 23  | 14  |

Let me know if there's anything else I can help you with!


### Chain-of-thought prompting

The results of question-answering can also be improved by prompting the LLM to provide intermediate steps.

**Exercise**: Using the following prompts, compare the answers of the "llama3-8b-8192" model (set seed=21). (If this model is no longer available choose a model with relatively few parameters.)

zero_shot_prompt = "How many s's are in the word 'success'?"

chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

In [225]:
prompts = ["How many s's are in the word 'success'?", "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."]

for p in prompts:
    response = client.chat.completions.create(
                messages = [{"role": "user", "content": p}],
                model = "llama3-8b-8192",
                seed = 21
)

    print('-'*50, '\n', p, '\n', response.choices[0].message.content)

-------------------------------------------------- 
 How many s's are in the word 'success'? 
 There are 2 s's in the word "success".
-------------------------------------------------- 
 How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn. 
 Let's go through each letter of the word "success" to count the number of s's:

1. S (first letter)
2. U
3. C
4. C
5. E
6. S
7. S

As we go through the letters, we find two S's (at positions 1 and 6) and three other letters. Therefore, the correct answer is:

There are 2 S's in the word "success".


## Comparison of LLMs

**Exercise**: Compare the performance of 2 LLMs by outputting the answers of the following questions into a dataframe.

    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",

Column headings:

Model Name | Question | Answer

In [245]:
pd.set_option('display.max_colwidth', None) # allows wide dataframes to be viewed
models = ["gemma2-9b-it", "llama-3.1-8b-instant"] #can edit this

prompts = ["Tell me a joke about data science.",
"How can one calculate 22 * 13 mentally?",
"Write a creative story about a baby learning to crawl."
]

results = {'Model Name': [], 'Question': [], 'Answer': []}

for model in models:
    for prompt in prompts:
        results['Model Name'].append(model)
        results['Question'].append(prompt)
        output = client.chat.completions.create(model = model, messages=[{"role": "user", "content": prompt}])
        results['Answer'].append(output.choices[0].message.content.strip())

df = pd.DataFrame(results)
df

,Model Name,Question,Answer
0,gemma2-9b-it,Tell me a joke about data science.,Why did the data scientist break up with the statistician? \n\nBecause they had too many arguments about p-values! 😂 \n\n\nLet me know if you'd like to hear another one! 😄
1,gemma2-9b-it,How can one calculate 22 * 13 mentally?,"Here's a way to calculate 22 * 13 mentally using a breakdown method:\n\n**1. Break down 22:** Think of 22 as (20 + 2)\n\n**2. Distribute:** Now you have (20 + 2) * 13. Distribute the 13:\n * 20 * 13 = 260 \n * 2 * 13 = 26\n\n**3. Add:** Add the two products: 260 + 26 = 286\n\n\n**Therefore, 22 * 13 = 286**"
2,gemma2-9b-it,Write a creative story about a baby learning to crawl.,"Bartholomew Buttercup, a cherub with an unruly tuft of blonde hair and a bottomless appetite for mashed sweet potatoes, was on a mission. His mission? To conquer the floor.\n\nHe couldn't quite understand why his legs refused to move as one. The world was a fascinating place, just out of reach – fluffy rugs begging to be crawled upon, a sparkly mobile dangling enticingly, and the elusive comfort of Mom's lap. \n\nOne sunny afternoon, while his chubby fingers tugged at a dangling teething toy, an idea, bright and unyielding as the sun shining through the nursery window, struck Bartholomew. He pushed himself up, his eyebrows furrowing in concentration, and let out a triumphant gurgle. This, he decided, was the key.\n\nFor the next few hours, Bartholomew ""skated"" – a wobbly, jerky motion fueled by desperation and mashed sweet potato-induced energy. He propelled himself across the floor, arms flailing like windmill wings, legs kicking blindly. \n\nEach failed attempt resulted in a frustrating topple, met with tears that were swiftly soothed by a cooing melody from Mom, followed by gentle pats and encouraging ""good tries."" \n\nFrustration was brewing. He wanted to be like his older sister, Lily, who glided across the floor like a gazelle on ice skates. But Bartholomew wasn't a gazelle. He was a Buttercup, and Buttercups, he decided, were meant to conquer the world, one wobbly crawl at a time.\n\nHe took a deep breath, remembering the way his sister moved. He watched her, his big brown eyes full of determination. \n\nSlowly, methodically, he began to coordinate his movements. One hand reached out, pushing against the rug. The other, mirroring the action, found purchase on the wooden floor. His legs, finally understanding their role, pushed down with surprising strength. \n\nHe felt a surge of triumph as he moved – not a jerky skate, but a smooth, controlled crawl. He was moving! He was conquering!\n\nA roar of glee erupted from his tiny throat. His legs pumped, propelling him across the room. He was a speedboat on a sunny sea, navigating the world with newfound freedom.\n\nHe stopped short, realizing his destination. Mom sat on the floor, a book open in her lap, humming a soft tune. \n\nWith a final burst of energy, Bartholomew reached her, collapsing into her open arms. \n\n""Yay, Bartholomew!"" Mom exclaimed, her voice filled with pride. ""You crawled!""\n\nBartholomew gurgled happily, his face flushed with exertion and triumph. He had conquered the floor. But more importantly, he had conquered the doubt, proving to himself that even the smallest Buttercup could achieve anything they set their mind to. The world, now within reach, beckoned. And Bartholomew Buttercup, he was ready."
3,llama-3.1-8b-instant,Tell me a joke about data science.,"Why did the data scientist's cat join a band?\n\nBecause it wanted to be the purr-cussionist and help with the data correlation, but it ended up having a regression problem with its rhythm."
4,llama-3.1-8b-instant,How can one calculate 22 * 13 mentally?,"To mentally calculate 22 * 13, you can use the method of breaking down the multiplication into simpler operations. \n\nHere's one way to do it:\n\n1. Break down 13 into (10 + 3)\n2. Multiply 22 by 10: 22 * 10 = 220\n3. Multiply 22 by 3: 22 * 3 = 66\n4. Add the two resu

### Bonus

See if you can prompt an LLM to perform sentiment analysis (output 'Positive' or 'Negative' only) on a given piece of text.

In [265]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": 'Identify the sentiment of "That movie was interesting." Output positive or negative only.'},
    ]
)

print(response.choices[0].message.content.strip())

Positive.


## Conclusion

We worked with a few Large Language Models (LLMs) using Groq and experimented with prompting for summarisation, text completion and question-answering tasks.

We also explored controlling the randomness (creativity) of output through the temperature setting and tried different types of prompting to achieve desired forms of output.

## References
1. [Groq's prompting guide](https://console.groq.com/docs/prompting)
2. [Groq's playground](https://console.groq.com/playground)



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



